In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [23]:
mem_data = pd.read_csv('../input/mem_data.csv')
mem_tr = pd.read_csv('../input/transactions.csv')
song_info = pd.read_csv('../input/songs.csv')
mem_tr_song = mem_tr.merge(song_info, how='left')

In [25]:
tr = mem_data.merge(mem_tr_song, how='left')

In [14]:
mem_tr.groupby('user_id')['song_id'].count()[:5]

user_id
1    220
4    874
5    393
7    224
8     62
Name: song_id, dtype: int64

In [9]:
mem_data[mem_data['gender']=='unknown']['gender'].count()

3720

In [16]:
mem_data.head()

,user_id,city,age,reg_method,reg_date,ex_date,gender
0,13827,14,29,9,20130112,20180203,female
1,6618,13,19,4,20160313,20180106,female
2,7615,22,33,9,20060314,20170930,female
3,15535,13,48,9,20110402,20171003,female
4,17708,18,20,4,20160102,20161105,female


In [17]:
song_info.head()

,song_id,length,genre,artist,composer,lyricist,language,isrc
0,5981,273554,465,330,1787,1533,3,TWC950206108
1,5767,228623,465,633,484,1057,3,TWA531398021
2,8128,257602,458,777,1916,2115,3,TWAY31508002
3,1992,199157,458,548,1011,1904,3,TWA471596001
4,2665,246015,465,671,1407,1362,3,TWK231306302


In [27]:
mem_tr_song.head()

,user_id,song_id,rec_loc,rec_screen,entry,listen,length,genre,artist,composer,lyricist,language,isrc
0,5,5,discover,Online playlist more,online-playlist,0,219150,465,55,960,520,17,JPB601402020
1,7,7,search,Search,song,1,320888,465,785,389,1078,3,TWA531530004
2,14,14,discover,Online playlist more,song-based-playlist,0,280868,465,603,1490,1449,3,TWA211650101
3,22,22,my library,Local playlist more,local-library,0,232907,444,66,570,517,31,TWA531600752
4,24,24,search,Online playlist more,online-playlist,0,296437,458,499,1384,1576,3,TWA470713001


In [80]:
# 차원축소 매소드 
from sklearn.decomposition import PCA

def dummy_to_pca(tr, column_name:str, features) :
    max_seq = 300
    max_d = 15
    col_count = tr.groupby(column_name)[column_name].count()
    if len(col_count) > max_seq:
        tops = col_count.sort_values(ascending=False)[0:max_seq].index
        f =tr.loc[tr[column_name].isin(tops)][['user_id', column_name]]
    else:
        tops = col_count.index
        f =tr[['user_id', column_name]]
    f = pd.get_dummies(f, columns=[column_name])  # This method performs One-hot-encoding
    f = f.groupby('user_id').mean()
    if len(tops) < max_d:
        max_d = len(tops)
    pca = PCA(n_components=max_d)
    pca.fit(f)
    cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합
    #print(cumsum)
    num_d = np.argmax(cumsum >= 0.99) + 1 # 분산의 설명량이 99%이상 되는 차원의 수
    if num_d == 1:
        num_d = max_d
    pca = PCA(n_components=num_d)    
    result = pca.fit_transform(f)
    result = pd.DataFrame(result)
    result.columns = [column_name + '_' + str(column) for column in result.columns]
    result.index = f.index
    return pd.concat([features, result], axis=1, join_axes=[features.index])
#f = dummy_to_pca(tr, 'team_nm', f)
#f

In [ ]:
tr = mem_data[['user_id', 'city', 'reg_method', 'gender']] 
tr.index = tr.user_id
# 경과일
tr.loc['R_DAY'] = (pd.to_datetime('2017-12-31') - pd.to_datetime(mem_data.reg_date, format='%Y%m%d')).dt.days
f = pd.to_datetime(mem_data.ex_date, format='%Y%m%d')
tr['E_DAY'] = (f.max() - f).dt.days

tr_gb = mem_tr_song.groupby('user_id').agg({'song_id':[('song_count','count')], 
                                    'listen':[('listen_sum', 'sum'), 
                                              ('listen_mean', 'mean' )],
                                    'genre': [('genre_nunique', 'nunique')],
                                    'artist': [('artist_nunique', 'nunique')],
                                    'language': [('language_nunique', 'nunique')],
                                   })
tr_gb.columns = tr_gb.columns.get_level_values(1)
tr_gb['user_id'] = tr_gb.index

tr = tr.merge(tr_gb,  how="left")
tr = dummy_to_pca(mem_tr_song, 'rec_loc', tr)
tr = dummy_to_pca(mem_tr_song, 'rec_screen', tr)
tr = dummy_to_pca(mem_tr_song, 'entry', tr)
tr = dummy_to_pca(mem_tr_song, 'genre', tr)
tr = dummy_to_pca(mem_tr_song, 'artist', tr)
tr = dummy_to_pca(mem_tr_song, 'composer', tr)
tr = dummy_to_pca(mem_tr_song, 'lyricist', tr)
tr = dummy_to_pca(mem_tr_song, 'language', tr)
tr.head()

C:\Users\south\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\south\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\south\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: 'user_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
